In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-large')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large')

In [ ]:
model.config

In [ ]:
prompt = 'How to install python package as command',
# prompt = 'How to prepare well done steak'

inputs = tokenizer(prompt, return_tensors='pt')
outputs = model.generate(
    **inputs,
    min_length=256,
    max_new_tokens=512,
    length_penalty=2,
    num_beams=16,
    no_repeat_ngram_size=2,
    temperature=0.8,
    top_k=150,
    top_p=0.9,
    early_stopping=True
)
outputs_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
outputs_text